Calculate Tractor Trips and movements

- Get a list of all direction ideintfied images (mike and Neil)
- function
    - find the first unclaimed image
    - if mike to > find neil to within time period > give both same trip id > if cant mark as unclaimed
    - if mike from find neil from in time period and give same trip id > if cant mark as unclaimed
    
    - if neil to find mike to in time period > if not mark as unclaimed
    - if neil from find mike from in time period > if not mark as unclaimed
    
    - Result - all jined up trip
        - combine trip tin movements > math to and from trip in righ ttime period

    - unclaimed
        - match unclaimed neils as not via mike
        - should be many unclaimed mike      



In [753]:
#cameras
MIKE = 1
NEIL = 2

#direction
TO = 1
FROM = 0

#status
NOT_CLAIMED = 0
CLAIMED = 1
ORPHAN = 2

In [754]:
import pandas as pd
import psycopg2
import db_config as creds
import os
import time
import datetime
from datetime import datetime  
from datetime import timedelta  

from PIL import Image

In [755]:
class tractor:
  def __init__(self, filename, datetaken, cameraid,cameraname,direction):
    self.datetaken = datetaken
    self.filename = filename
    self.cameraname = cameraname
    self.cameraid = cameraid
    self.direction = direction
        
   

## Database Functions

In [756]:
def connect():
    
    # Set up a connection to the postgres server.
    conn_string = "host="+ creds.PGHOST +" port="+ "5432" +" dbname="+ creds.PGDATABASE +" user=" + creds.PGUSER \
                  +" password="+ creds.PGPASSWORD
         
    conn = psycopg2.connect(conn_string)

    # Create a cursor object
    cursor = conn.cursor()
  
    #print ("connected!")
    return conn, cursor

def disconnect(conn,cursor):
    
    cursor.close()
    conn.close()
    #print ("disconnected!")

In [757]:
def getTractors():

    # Connecting to DB
    conn, cursor = connect()
    
    #get list of all tractor images with a direction
    dat = pd.read_sql(('select filename, datetaken_adjusted, cameraid, cameraname, true_direction from images '
                     'where datetaken BETWEEN %(dstart)s AND %(dfinish)s and true_direction is not null order by datetaken_adjusted asc'),
                   conn,params={"dstart":datetime(2020,5,8,0,0),"dfinish":datetime(2020,5,9,0,0)})

       
    dat["claimed"] = NOT_CLAIMED
    dat["tripid"] = 0
    dat["movementid"] = 0
    
    # Disconnect from DB
    disconnect(conn, cursor)
    
    print(dat)
    return dat

def findTrips(tractors):

    print("jamie")
    unclaimed = len(tractors[tractors['claimed'] == 0])
                    
                    
    while (unclaimed > 0):
    
        print(unclaimed) 
    
        #find next unclaimed tractor
        next_t = nextUnclaimed(tractors)
   
        #look for a related on either mike or neil           
        related_t = lookForRelated(next_t, tractors)
    
        #Get the next trip id
        tripid = tractors["tripid"].max() + 1
    
        #orphan, we cant find a matching trip image, set trip id, set status      
        if (next_t.filename == related_t.filename):
                    
            tractors.loc[tractors.filename == next_t.filename,'claimed'] = 1
            tractors.loc[tractors.filename == next_t.filename,'tripid'] = tripid 
            #print(tractors.head())
        
        else:
        
            #update image
            tractors.loc[tractors.filename == next_t.filename, 'claimed'] = 1
            tractors.loc[tractors.filename == next_t.filename,'tripid'] = tripid 
        
            #update related image  
            tractors.loc[tractors.filename == related_t.filename,'claimed'] = 1
            tractors.loc[tractors.filename == related_t.filename,'tripid'] = tripid 
    
        print("Original",next_t.filename,"Related to",related_t.filename)
        unclaimed = len(tractors[tractors['claimed'] == 0])
    else:
        print(tractors)      
    
   
def nextUnclaimed(tractors):

    #find the next tractor in the dataframe that is unclames
    t = tractors[tractors.claimed == 0].iloc[0]
    #print (t)
    nextTractor = tractor(t[0],t[1],t[2],t[3],t[4])
   
    return nextTractor

def lookForRelated(t, tractors):  

    #print (t.filename)
    time_min, time_max = getTimeWindow(t)
    
    #mike
    if (t.cameraid == MIKE):
        
        # look for related neil to image 
        try:
            related_t = tractors[
                             (tractors.filename != t.filename)
                             & (tractors.claimed == 0) 
                             & (tractors.cameraid == NEIL)
                             & (tractors.true_direction == t.direction)
                             & (tractors['datetaken_adjusted'] >= time_min)
                             & (tractors['datetaken_adjusted'] <= time_max)].iloc[0]
      

        
        except:
            print("No matching Neil image")
            related_t = t #just return the original
        
    else:    
        
        try:
            related_t = tractors[
                             (tractors.filename != t.filename)
                             & (tractors.claimed == 0)
                             & (tractors.cameraid == MIKE)   
                             & (tractors.true_direction == t.direction) 
                             & (tractors['datetaken_adjusted'] >= time_min)
                             & (tractors['datetaken_adjusted'] <= time_max)].iloc[0]
        
        except:
            print("No matching mike image")
            related_t = t #just return the original
        
    return related_t
    
   
def getTimeWindow(t):
    
    #return min and max time range
    # it takes between 4.5 and 8 minutes to get between mike and neil 
    mins_min, mins_max = 4.5, 8
    
    #if mike to, then add time for neil
    if (t.cameraid == MIKE and t.direction == TO):
        minTime = t.datetaken + timedelta(minutes=mins_min)
        maxTime = t.datetaken + timedelta(minutes=mins_max)
    
    elif (t.cameraid == MIKE and t.direction == FROM):
        minTime = t.datetaken - timedelta(minutes=mins_min)
        maxTime = t.datetaken - timedelta(minutes=mins_max) 
    
    elif (t.cameraid == NEIL and t.direction == TO):
        minTime = t.datetaken - timedelta(minutes=mins_min)
        maxTime = t.datetaken - timedelta(minutes=mins_max) 
    
    elif (t.cameraid == NEIL and t.direction == FROM):
        minTime = t.datetaken + timedelta(minutes=mins_min)
        maxTime = t.datetaken + timedelta(minutes=mins_max) 
    
    
    #print("taken-",t.datetaken,"min-",minTime,"max-",maxTime)
    
    return minTime, maxTime    
    


In [758]:
#get list of all tractors
df_tractors = getTractors()

#find trips
#df_tractor_trips = findTrips(df_tractors)
findTrips(df_tractors)

#find movements = findMovements()
#df_tractor_movs = findMovements(df_tractor_trips)


#update database
#Updatedb(df_tractor_movs)


         filename  datetaken_adjusted  cameraid cameraname  true_direction  \
0    14PT1460.JPG 2020-05-08 06:33:56         2       Neil               1   
1    04Mk2351.JPG 2020-05-08 06:36:48         1       Mike               1   
2    14PT1462.JPG 2020-05-08 06:43:20         2       Neil               1   
3    14PT1464.JPG 2020-05-08 06:45:52         2       Neil               0   
4    04Mk2355.JPG 2020-05-08 06:50:50         1       Mike               0   
..            ...                 ...       ...        ...             ...   
493  15PT1717.JPG 2020-05-08 21:47:20         2       Neil               0   
494  04Mk3851.JPG 2020-05-08 21:50:54         1       Mike               0   
495  04Mk3855.JPG 2020-05-08 21:51:00         1       Mike               0   
496  04Mk3859.JPG 2020-05-08 21:52:46         1       Mike               0   
497  15PT1719.JPG 2020-05-08 22:17:22         2       Neil               0   

     claimed  tripid  movementid  
0          0       0        

Original 04Mk3032.JPG Related to 04Mk3032.JPG
311
Original 04Mk3036.JPG Related to 15PT1558.JPG
309
No matching Neil image
Original 04Mk3041.JPG Related to 04Mk3041.JPG
308
Original 15PT1560.JPG Related to 04Mk3053.JPG
306
No matching mike image
Original 15PT1561.JPG Related to 15PT1561.JPG
305
No matching Neil image
Original 04Mk3046.JPG Related to 04Mk3046.JPG
304
Original 04Mk3049.JPG Related to 15PT1562.JPG
302
No matching Neil image
Original 04Mk3058.JPG Related to 04Mk3058.JPG
301
Original 15PT1563.JPG Related to 04Mk3079.JPG
299
No matching Neil image
Original 04Mk3065.JPG Related to 04Mk3065.JPG
298
Original 04Mk3069.JPG Related to 15PT1565.JPG
296
Original 04Mk3074.JPG Related to 15PT1567.JPG
294
No matching mike image
Original 15PT1564.JPG Related to 15PT1564.JPG
293
Original 15PT1566.JPG Related to 04Mk3089.JPG
291
No matching Neil image
Original 04Mk3082.JPG Related to 04Mk3082.JPG
290
No matching Neil image
Original 04Mk3085.JPG Related to 04Mk3085.JPG
289
Original 15PT156

Original 15PT1661.JPG Related to 04Mk3592.JPG
105
Original 15PT1664.JPG Related to 04Mk3596.JPG
103
No matching mike image
Original 15PT1665.JPG Related to 15PT1665.JPG
102
Original 15PT1666.JPG Related to 04Mk3604.JPG
100
Original 15PT1667.JPG Related to 04Mk3608.JPG
98
No matching Neil image
Original 04Mk3599.JPG Related to 04Mk3599.JPG
97
No matching mike image
Original 15PT1668.JPG Related to 15PT1668.JPG
96
No matching Neil image
Original 04Mk3611.JPG Related to 04Mk3611.JPG
95
No matching mike image
Original 15PT1669.JPG Related to 15PT1669.JPG
94
Original 04Mk3616.JPG Related to 15PT1670.JPG
92
No matching Neil image
Original 04Mk3621.JPG Related to 04Mk3621.JPG
91
No matching mike image
Original 15PT1671.JPG Related to 15PT1671.JPG
90
Original 15PT1672.JPG Related to 04Mk3634.JPG
88
Original 15PT1673.JPG Related to 04Mk3638.JPG
86
No matching Neil image
Original 04Mk3625.JPG Related to 04Mk3625.JPG
85
Original 04Mk3631.JPG Related to 15PT1674.JPG
83
No matching mike image
Origi